# Attempt to use ConcreteStructs.jl with Parameters.jl

* Gen Kuroki
* 2021-09-06

I have independently tried to make ConcreteStructs.jl and Parameters.jl work together well.

__Conclusion:__ It is possible to do so by making the following two changes.

* Change `@concrete` not creating the inner constructor, so that the `Foo{__T_a, __T_b, __T_c}(a, b, c)`-type default constructor will be defined.
* Change `@with_kw` expanding macros in the argument, like `Base.@kwdef`.

Then `@concrete` works well with `@with_kw` and more completely with `Base.@kwdef`.

See below for details.

In [1]:
VERSION

v"1.6.2"

In [2]:
using ConcreteStructs
using Parameters

macro macroexpand_rmln(code)
    :(macroexpand($__module__, $(QuoteNode(code)), recursive=true) |>
        Base.remove_linenums!)
end

@macroexpand_rmln (macro with 1 method)

## Plain struct

In [3]:
struct Foo{A, B, C}
    a::A
    b::B
    c::C
end

In [4]:
methods(Foo)

# 1 method for type constructor:
[1] Foo(a::A, b::B, c::C) where {A, B, C} in Main at In[3]:2

In [5]:
methods(Foo{1,2,3})

# 1 method for type constructor:
[1] Foo{A, B, C}(a, b, c) where {A, B, C} in Main at In[3]:2

The default constructor `Foo{A, B, C}(a, b, c)` is defined.

## @concrete struct

`@concrete` removes the `Foo_concrete{__T_a, __T_b, __T_c}(a, b, c)`-type default constructor.

In [6]:
@concrete struct Foo_concrete
    a
    b
    c
end

In [7]:
methods(Foo_concrete)

# 1 method for type constructor:
[1] Foo_concrete(a::__T_a, b::__T_b, c::__T_c) where {__T_a, __T_b, __T_c} in Main at D:\.julia\packages\ConcreteStructs\7Lv7u\src\ConcreteStructs.jl:142

In [8]:
methods(Foo_concrete{1,2,3})

# 0 methods for type constructor:

`Foo_concrete{__T_a, __T_b, __T_c}(a, b, c)` is not defined because only the inner constructor `Foo_concrete(a::__T_a, b::__T_b, c::__T_c)` is defined.

In [9]:
@macroexpand_rmln @concrete struct Foo_concrete
    a
    b
    c
end

:(struct Foo_concrete{__T_a, __T_b, __T_c} <: Any
      a::__T_a
      b::__T_b
      c::__T_c
      function Foo_concrete(a::__T_a, b::__T_b, c::__T_c) where {__T_a, __T_b, __T_c}
          return new{__T_a, __T_b, __T_c}(a, b, c)
      end
  end)

## Base.@kwdef concrete struct

In [10]:
Base.@kwdef @concrete struct Foo_kwdef_concrete
    a = 1
    b = 2.0
    c = "three"
end

In [11]:
Foo_kwdef_concrete()

Foo_kwdef_concrete{Int64, Float64, String}(1, 2.0, "three")

In [12]:
Foo_kwdef_concrete{Int, Float64, String}(a = 4, b = 5.0, c = "six")

LoadError: MethodError: no method matching Foo_kwdef_concrete{Int64, Float64, String}(::Int64, ::Float64, ::String)

The reason for this error is that `Foo_kwdef_concrete{__T_a, __T_b, __T_c}(a, b, c)` is not defined.

In [13]:
methods(Foo_kwdef_concrete)

# 2 methods for type constructor:
[1] Foo_kwdef_concrete(; a, b, c) in Main at util.jl:461
[2] Foo_kwdef_concrete(a::__T_a, b::__T_b, c::__T_c) where {__T_a, __T_b, __T_c} in Main at D:\.julia\packages\ConcreteStructs\7Lv7u\src\ConcreteStructs.jl:142

In [14]:
methods(Foo_kwdef_concrete{1,2,3})

# 1 method for type constructor:
[1] Foo_kwdef_concrete{__T_a, __T_b, __T_c}(; a, b, c) where {__T_a, __T_b, __T_c} in Main at util.jl:462

In [15]:
@macroexpand_rmln Base.@kwdef @concrete struct Foo_kwdef_concrete
    a
    b
    c
end

quote
    begin
        $(Expr(:meta, :doc))
        struct Foo_kwdef_concrete{__T_a, __T_b, __T_c} <: Any
            a::__T_a
            b::__T_b
            c::__T_c
            function Foo_kwdef_concrete(a::__T_a, b::__T_b, c::__T_c) where {__T_a, __T_b, __T_c}
                return new{__T_a, __T_b, __T_c}(a, b, c)
            end
        end
    end
    begin
        Foo_kwdef_concrete(; a, b, c) = begin
                Foo_kwdef_concrete(a, b, c)
            end
        (Foo_kwdef_concrete{__T_a, __T_b, __T_c}(; a, b, c) where {__T_a, __T_b, __T_c}) = begin
                Foo_kwdef_concrete{__T_a, __T_b, __T_c}(a, b, c)
            end
    end
end

## @with_kw @concrete struct causes error

In [16]:
@with_kw @concrete struct Foo_with_kw_concrete
    a = 1
    b = 2.0
    c = "three"
end

LoadError: LoadError: Only works on type-defs or named tuples.
Make sure to have a space after `@with_kw`, e.g. `@with_kw (a=1,)
Also, make sure to use a trailing comma for single-field NamedTuples.

in expression starting at In[16]:1

[The first line of `Base.@kwdef`](https://github.com/JuliaLang/julia/blob/4931faa34a8a1c98b39fb52ed4eb277729120128/base/util.jl#L455) expands macros in the argument expression:

```julia
    expr = macroexpand(__module__, expr) # to expand @static
```

This is what makes `Base.@kwdef @concrete struct` possible.  Make a similar change to `Parameters.@with_kw`.

## Change @with_kw expanding macros in the argument

In [17]:
@eval Parameters macro with_kw(typedef)
    typedef = macroexpand(__module__, typedef) # inserted
    return esc(with_kw(typedef, __module__, true))
end

@with_kw (macro with 2 methods)

In [18]:
@with_kw @concrete struct Foo_with_kw_concrete
    a = 1
    b = 2.0
    c = "three"
end

Foo_with_kw_concrete

Okay, it seems to have worked.  But...

In [19]:
methods(Foo_with_kw_concrete)

# 5 methods for type constructor:
[1] Foo_with_kw_concrete(; a, b, c) in Main at D:\.julia\packages\Parameters\cGriM\src\Parameters.jl:520
[2] Foo_with_kw_concrete(pp::Foo_with_kw_concrete; kws...) in Main at D:\.julia\packages\Parameters\cGriM\src\Parameters.jl:530
[3] Foo_with_kw_concrete(pp::Foo_with_kw_concrete, di::AbstractDict) in Main at D:\.julia\packages\Parameters\cGriM\src\Parameters.jl:533
[4] Foo_with_kw_concrete(pp::Foo_with_kw_concrete, di::Tuple{Symbol, Any}...) in Main at D:\.julia\packages\Parameters\cGriM\src\Parameters.jl:534
[5] Foo_with_kw_concrete(a::__T_a, b::__T_b, c::__T_c) where {__T_a, __T_b, __T_c} in Main at D:\.julia\packages\Parameters\cGriM\src\Parameters.jl:501

In [20]:
methods(Foo_with_kw_concrete{1,2,3})

# 1 method for type constructor:
[1] Foo_with_kw_concrete{__T_a, __T_b, __T_c}(; a, b, c) where {__T_a, __T_b, __T_c} in Main at D:\.julia\packages\Parameters\cGriM\src\Parameters.jl:466

In [21]:
Foo_with_kw_concrete()

LoadError: MethodError: no method matching Foo_with_kw_concrete{Int64, Float64, String}(::Int64, ::Float64, ::String)

The reason for this error is that `Foo_with_kw_concrete{__T_a, __T_b, __T_c}(a, b, c)` is not defined again.

In [22]:
@macroexpand_rmln @with_kw @concrete struct Foo_with_kw_concrete
    a = 1
    b = 2.0
    c = "three"
end

quote
    begin
        $(Expr(:meta, :doc))
        struct Foo_with_kw_concrete{__T_a, __T_b, __T_c} <: Any
            "Default: 1"
            a::__T_a
            "Default: 2.0"
            b::__T_b
            "Default: three"
            c::__T_c
            (Foo_with_kw_concrete{__T_a, __T_b, __T_c}(; a = 1, b = 2.0, c = "three") where {__T_a, __T_b, __T_c}) = begin
                    Foo_with_kw_concrete{__T_a, __T_b, __T_c}(a, b, c)
                end
            function Foo_with_kw_concrete(a::__T_a, b::__T_b, c::__T_c) where {__T_a, __T_b, __T_c}
                return new{__T_a, __T_b, __T_c}(a, b, c)
            end
        end
    end
    (Foo_with_kw_concrete(a::__T_a, b::__T_b, c::__T_c) where {__T_a, __T_b, __T_c}) = begin
            Foo_with_kw_concrete{__T_a, __T_b, __T_c}(a, b, c)
        end
    Foo_with_kw_concrete(; a = 1, b = 2.0, c = "three") = begin
            Foo_with_kw_concrete(a, b, c)
        end
    begin
        Foo_with_kw_concrete(pp::Foo_with_kw

## Change `@concrete` not creating the inner constructor

Change `ConcreteStructs._concretize(expr)` not creating the inner constructor.

In [23]:
@eval ConcreteStructs function _concretize(expr)
    expr isa Expr && expr.head == :struct || error("Invalid usage of @concrete")
    
    is_mutable = expr.args[1]
    struct_name, type_params, super = _parse_head(expr.args[2])
    line_tuples = _parse_line.(expr.args[3].args)
    lines = first.(line_tuples)
    type_params_full = (type_params..., filter(x -> x!==nothing, last.(line_tuples))...)

    struct_type = if length(type_params_full) == 0
        struct_name
    else
        Expr(:curly, struct_name, type_params_full...)
    end

    head = Expr(:(<:), struct_type, super)
    # constructor_expr = _make_constructor(struct_name, type_params, type_params_full, lines)
    # body = Expr(:block, lines..., constructor_expr)
    body = Expr(:block, lines...)
    struct_expr = Expr(:struct, is_mutable, head, body)
    
    return struct_expr, struct_name, type_params
end

_concretize (generic function with 1 method)

In [24]:
@macroexpand_rmln @concrete struct Bar_concrete
    a = 1
    b = 2.0
    c = "three"
end

:(struct Bar_concrete{__T_a, __T_b, __T_c} <: Any
      a::__T_a = 1
      b::__T_b = 2.0
      c::__T_c = "three"
  end)

The inner constructor has been deleted.

## @concrete works well with @with_kw

In [25]:
@with_kw @concrete struct Bar_with_kw_concrete
    a = 1
    b = 2.0
    c = "three"
end

Bar_with_kw_concrete

In [26]:
methods(Bar_with_kw_concrete)

# 5 methods for type constructor:
[1] Bar_with_kw_concrete(; a, b, c) in Main at D:\.julia\packages\Parameters\cGriM\src\Parameters.jl:520
[2] Bar_with_kw_concrete(pp::Bar_with_kw_concrete; kws...) in Main at D:\.julia\packages\Parameters\cGriM\src\Parameters.jl:530
[3] Bar_with_kw_concrete(pp::Bar_with_kw_concrete, di::AbstractDict) in Main at D:\.julia\packages\Parameters\cGriM\src\Parameters.jl:533
[4] Bar_with_kw_concrete(pp::Bar_with_kw_concrete, di::Tuple{Symbol, Any}...) in Main at D:\.julia\packages\Parameters\cGriM\src\Parameters.jl:534
[5] Bar_with_kw_concrete(a::__T_a, b::__T_b, c::__T_c) where {__T_a, __T_b, __T_c} in Main at D:\.julia\packages\Parameters\cGriM\src\Parameters.jl:501

In [27]:
methods(Bar_with_kw_concrete{1,2,3})

# 2 methods for type constructor:
[1] Bar_with_kw_concrete{__T_a, __T_b, __T_c}(; a, b, c) where {__T_a, __T_b, __T_c} in Main at D:\.julia\packages\Parameters\cGriM\src\Parameters.jl:466
[2] Bar_with_kw_concrete{__T_a, __T_b, __T_c}(a, b, c) where {__T_a, __T_b, __T_c} in Main at D:\.julia\packages\Parameters\cGriM\src\Parameters.jl:478

The default constructor `Bar_with_kw_concrete{__T_a, __T_b, __T_c}(a, b, c)` is implicitly defined.

In [28]:
Bar_with_kw_concrete()

Bar_with_kw_concrete{Int64, Float64, String}
  a: Int64 1
  b: Float64 2.0
  c: String "three"


In [29]:
Bar_with_kw_concrete(c = '3')

Bar_with_kw_concrete{Int64, Float64, Char}
  a: Int64 1
  b: Float64 2.0
  c: Char '3'


In [30]:
Bar_with_kw_concrete{Int, Float64, String}(a = 4, b = 5.0, c = "six")

Bar_with_kw_concrete{Int64, Float64, String}
  a: Int64 4
  b: Float64 5.0
  c: String "six"


In [31]:
Bar_with_kw_concrete(4, 5.0, "six")

Bar_with_kw_concrete{Int64, Float64, String}
  a: Int64 4
  b: Float64 5.0
  c: String "six"


`ConcreteStructs.@concrete` works well with `Parameters.@with_kw`!

In [32]:
@macroexpand_rmln @with_kw @concrete struct Bar_with_kw_concrete
    a = 1
    b = 2.0
    c = "three"
end

quote
    begin
        $(Expr(:meta, :doc))
        struct Bar_with_kw_concrete{__T_a, __T_b, __T_c} <: Any
            "Default: 1"
            a::__T_a
            "Default: 2.0"
            b::__T_b
            "Default: three"
            c::__T_c
            (Bar_with_kw_concrete{__T_a, __T_b, __T_c}(; a = 1, b = 2.0, c = "three") where {__T_a, __T_b, __T_c}) = begin
                    Bar_with_kw_concrete{__T_a, __T_b, __T_c}(a, b, c)
                end
            (Bar_with_kw_concrete{__T_a, __T_b, __T_c}(a, b, c) where {__T_a, __T_b, __T_c}) = begin
                    new{__T_a, __T_b, __T_c}(a, b, c)
                end
        end
    end
    (Bar_with_kw_concrete(a::__T_a, b::__T_b, c::__T_c) where {__T_a, __T_b, __T_c}) = begin
            Bar_with_kw_concrete{__T_a, __T_b, __T_c}(a, b, c)
        end
    Bar_with_kw_concrete(; a = 1, b = 2.0, c = "three") = begin
            Bar_with_kw_concrete(a, b, c)
        end
    begin
        Bar_with_kw_concrete(pp::Bar_with_

## @concrete works well more completely with Base.@kwdef

In [33]:
Base.@kwdef @concrete struct Bar_kwdef_concrete
    a = 1
    b = 2.0
    c = "three"
end

In [34]:
Bar_kwdef_concrete()

Bar_kwdef_concrete{Int64, Float64, String}(1, 2.0, "three")

In [35]:
Bar_kwdef_concrete(c = '3')

Bar_kwdef_concrete{Int64, Float64, Char}(1, 2.0, '3')

In [36]:
Bar_kwdef_concrete{Int, Float64, String}(a = 4, b = 5.0, c = "six") # not error

Bar_kwdef_concrete{Int64, Float64, String}(4, 5.0, "six")

In [37]:
Bar_kwdef_concrete(4, 5.0, "six")

Bar_kwdef_concrete{Int64, Float64, String}(4, 5.0, "six")

`ConcreteStructs.@concrete` works more completely well with `Base.@kwdef`!

In [38]:
@macroexpand_rmln Base.@kwdef @concrete struct Bar_kwdef_concrete
    a = 1
    b = 2.0
    c = "three"
end

quote
    begin
        $(Expr(:meta, :doc))
        struct Bar_kwdef_concrete{__T_a, __T_b, __T_c} <: Any
            a::__T_a
            b::__T_b
            c::__T_c
        end
    end
    begin
        Bar_kwdef_concrete(; a = 1, b = 2.0, c = "three") = begin
                Bar_kwdef_concrete(a, b, c)
            end
        (Bar_kwdef_concrete{__T_a, __T_b, __T_c}(; a = 1, b = 2.0, c = "three") where {__T_a, __T_b, __T_c}) = begin
                Bar_kwdef_concrete{__T_a, __T_b, __T_c}(a, b, c)
            end
    end
end